In [ ]:
# This code calculates the vertical DEM shift using the Nuth and Kääb point cloud coregistration
# activate the xdem_dev environment

import geoutils as gu
import numpy as np
from numpy.linalg import inv
import pandas as pd

import xdem

In [ ]:
# **Inputs**
path = 'Sites/RCEW/DEMs/'
meanmin = 'min'

icesat_file = 'RCEW-ICESat2-ATL03-min'
DEM_file = 'RCEW_1m_WGS84UTM11_WGS84'
slope_file = 'RCEW_1m_WGS84UTM11_WGS84-slope'
aspect_file = 'RCEW_1m_WGS84UTM11_WGS84-aspect'

In [ ]:
# **files**
reference_dem = xdem.DEM(path + icesat_file + '.tif')
dem_to_be_aligned = xdem.DEM(path + DEM_file + '.tif')
dem_elev = xdem.DEM(path + DEM_file + '.tif')
dem_slope = xdem.DEM(path + slope_file + '.tif')
dem_aspect = xdem.DEM(path + aspect_file + '.tif')

In [ ]:
## **difference before coregistration**
#diff_before = reference_dem - dem_to_be_aligned
#diff_before.show(cmap="coolwarm_r", vmin=-1, vmax=1, cbar_title="Elevation change (m)")

## Coreggister DEM to IS2 (simpler method)

In [ ]:
# **Nuth and Kääb coregistration**

#nuth_kaab = xdem.coreg.NuthKaab()
#nuth_kaab.fit(reference_dem,dem_to_be_aligned)

In [ ]:
# **apply coregistration shift**
#aligned_dem = nuth_kaab.apply(dem_elev)
#aligned_slope = nuth_kaab.apply(dem_slope)
#aligned_aspect = nuth_kaab.apply(dem_aspect)

In [ ]:
# **Plot the difference between DTMs**
#diff = dem_to_be_aligned - aligned_dem   
#diff.show(cmap="coolwarm_r", vmin=-1, vmax=1, cbar_title="Elevation change (m)")

In [ ]:
# **export coregistered DEMs**
#aligned_dem.save(path + DEM_file + '_' + meanmin + '_CoRegAlt.tif')
#aligned_slope.save(path + slope_file + '_' + meanmin +'_CoRegAlt.tif')
#aligned_aspect.save(path + aspect_file + '_' + meanmin +'_CoRegAlt.tif')

## Coreggister IS2 to DEM (more complicated method)

In [ ]:
# **Nuth and Kääb coregistration**
nuth_kaab = xdem.coreg.NuthKaab()

nuth_kaab.fit(dem_to_be_aligned, reference_dem)

In [ ]:
#save the transform as a matrix
a = nuth_kaab.to_matrix() 
a

In [ ]:
#Invert transphormation matrix
ainv = inv(a)
ainv

In [ ]:
# **Make new cogrestration from inverted transformation matrix**
nuth_kaab2= xdem.coreg.affine.AffineCoreg()
nuth_kaab2._meta["matrix"] = ainv #manually set transformation matrix to desired matrix
c = nuth_kaab2._meta.get("matrix") #double check the matrix
c

In [ ]:
# **apply coregistration shift**
aligned_dem = nuth_kaab2.apply(dem_elev)
aligned_slope = nuth_kaab2.apply(dem_slope)
aligned_aspect = nuth_kaab2.apply(dem_aspect)

In [ ]:
# **Plot the difference between DTMs**
# this can be used to check that the transformation did what you expect
diff = dem_to_be_aligned - aligned_dem
diff.show(cmap="coolwarm_r", vmin=-1, vmax=1, cbar_title="Elevation change (m)")

In [ ]:
# **export coregistered DEMs**
#aligned_dem.save(path + DEM_file + '_' + meanmin + '_CoRegAlt.tif')
#aligned_slope.save(path + slope_file + '_' + meanmin +'_CoRegAlt.tif')
#aligned_aspect.save(path + aspect_file + '_' + meanmin +'_CoRegAlt.tif')